# 一、下载NCBI序列和信息

In [ ]:
# from Bio import Entrez, SeqIO

# # 设置您的邮箱地址
# Entrez.email = "your_email@example.com"

# # 指定保存文件的目录
# save_directory = r"F:/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列"
# #save_directory = r"/mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列"
# # 定义函数以下载序列并提取特征信息
# def download_and_process_sequences(id_list):
#     for seq_id in id_list:
#         # 下载fasta序列
#         handle_fasta = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text")
#         record_fasta = SeqIO.read(handle_fasta, "fasta")
#         handle_fasta.close()

#         # 保存fasta文件
#         filename_fasta = f"{save_directory}\\{seq_id}.fasta"
#         SeqIO.write(record_fasta, filename_fasta, "fasta")
#         print(f"Downloaded and saved {filename_fasta}")

#         # 下载genbank记录以提取特征信息
#         handle_gb = Entrez.efetch(db="nucleotide", id=seq_id, rettype="gb", retmode="text")
#         record_gb = SeqIO.read(handle_gb, "genbank")
#         country = isolate = lat_lon = "Not Available"
#         for feature in record_gb.features:
#             if feature.type == "source":
#                 qualifiers = feature.qualifiers
#                 country = qualifiers.get("country", ["Not Available"])[0]
#                 isolate = qualifiers.get("isolate", ["Not Available"])[0]
#                 lat_lon = qualifiers.get("lat_lon", ["Not Available"])[0]
#                 break

#         reference_titles = ", ".join([ref.title for ref in record_gb.annotations.get("references", []) if ref.title])
#         with open(f"F:/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列/基本信息.txt", "a") as output_file:
#         #with open(f"/mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列/基本信息.txt", "a") as output_file:
#             output_file.write(f"{seq_id}\t{country}\t{isolate}\t{lat_lon}\t{reference_titles}\n")
#         print(f"{seq_id}\t信息已经打印！")
#         handle_gb.close()

# # 从同一个文件读取序列ID
# id_list = []
# with open("F:/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载NCBI.txt", "r") as file:
# #with open("/mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载NCBI.txt", "r") as file:

#     for line in file:
#         id_list.append(line.strip())

# # 调用函数
# download_and_process_sequences(id_list)

## 并行版本

In [2]:
from Bio import Entrez, SeqIO
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

# 设置您的邮箱地址
Entrez.email = "giantlinlinlin@gmail.com"

# 指定保存文件的目录
save_directory = r"/mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列"
os.makedirs(save_directory, exist_ok=True)

# 初始化日志文件
success_log_file = os.path.join(save_directory, "success_log.txt")
failure_log_file = os.path.join(save_directory, "failure_log.txt")

# 定义函数以下载序列并提取特征信息
def download_and_process_sequence(seq_id):
    try:
        # 下载fasta序列
        handle_fasta = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text")
        record_fasta = SeqIO.read(handle_fasta, "fasta")
        handle_fasta.close()

        # 保存fasta文件
        filename_fasta = f"{save_directory}\\{seq_id}.fasta"
        # SeqIO.write(record_fasta, filename_fasta, "fasta")
        print(f"Downloaded and saved {filename_fasta}")

        # 下载genbank记录以提取特征信息
        handle_gb = Entrez.efetch(db="nucleotide", id=seq_id, rettype="gb", retmode="text")
        record_gb = SeqIO.read(handle_gb, "genbank")
        country = isolate = lat_lon = "Not Available"
        for feature in record_gb.features:
            if feature.type == "source":
                qualifiers = feature.qualifiers
                country = qualifiers.get("country", ["Not Available"])[0]
                isolate = qualifiers.get("isolate", ["Not Available"])[0]
                lat_lon = qualifiers.get("lat_lon", ["Not Available"])[0]
                break

        reference_titles = ", ".join([ref.title for ref in record_gb.annotations.get("references", []) if ref.title])
        with open(f"/mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列/基本信息.txt", "a") as output_file:
            output_file.write(f"{seq_id}\t{country}\t{isolate}\t{lat_lon}\t{reference_titles}\n")
        print(f"{seq_id}\t信息已经打印！")
        handle_gb.close()

        # 如果成功，记录到成功日志
        with open(success_log_file, "a") as success_log:
            success_log.write(f"{seq_id}\n")

    except Exception as e:
        print(f"Error processing {seq_id}: {str(e)}")

        # 如果失败，记录到失败日志
        with open(failure_log_file, "a") as failure_log:
            failure_log.write(f"{seq_id}\t{str(e)}\n")

# 从同一个文件读取序列ID
id_list = []
with open("/mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载NCBI.txt", "r") as file:
    for line in file:
        id_list.append(line.strip())

# 使用线程池并行下载和处理序列
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(download_and_process_sequence, seq_id) for seq_id in id_list]

    for future in as_completed(futures):
        future.result()  # 检查是否有异常

print("所有序列处理完成。")


Downloaded and saved /mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列\MN849629.fasta
Downloaded and saved /mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列\MN849746.fasta
Downloaded and saved /mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列\MN849846.fasta
Error processing MN849658: HTTP Error 429: Too Many Requests
Downloaded and saved /mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列\MN849839.fasta
MN849746	信息已经打印！
MN849846	信息已经打印！
MN849629	信息已经打印！
MN849839	信息已经打印！
所有序列处理完成。


# 二、处理下载序列

In [1]:
import os
import re
from Bio import SeqIO

# 目标文件夹
folder_path = r"/mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列"

# 遍历目录下所有 fasta 文件
for filename in os.listdir(folder_path):
    if filename.lower().endswith((".fasta", ".fa")):
        file_path = os.path.join(folder_path, filename)
        
        # 临时保存修改后的记录
        modified_records = []
        
        with open(file_path, "r", encoding="utf-8") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                # 删除 ID 中的 `.1` `.2` `.3` 后缀
                original_id = record.id
                cleaned_id = re.sub(r"\.\d+$", "", record.id)
                record.id = cleaned_id
                record.name = cleaned_id  # 一般也一并更新 name
                record.description = cleaned_id  # 只保留 ID

                modified_records.append(record)

        # 写回原文件（覆盖）
        with open(file_path, "w", encoding="utf-8") as output_handle:
            SeqIO.write(modified_records, output_handle, "fasta")

        print(f"处理完成: {filename}")


处理完成: PV067297.fasta
处理完成: PV067298.fasta
处理完成: PV067299.fasta
处理完成: PV067300.fasta
处理完成: PV067301.fasta
处理完成: PV067302.fasta
处理完成: PV067303.fasta
处理完成: PV067304.fasta
处理完成: PV067305.fasta
处理完成: PV067306.fasta
处理完成: PV067307.fasta
处理完成: PV067308.fasta
处理完成: PV067309.fasta
处理完成: PV067310.fasta
处理完成: PV067311.fasta
处理完成: PV067312.fasta
处理完成: PV067313.fasta
处理完成: PV067314.fasta
处理完成: PV067315.fasta
处理完成: PV067316.fasta
处理完成: PV067317.fasta
处理完成: PV067318.fasta
处理完成: PV067319.fasta
处理完成: PV067320.fasta
处理完成: PV067321.fasta
处理完成: PV067322.fasta
处理完成: PV067323.fasta
处理完成: PV067324.fasta
处理完成: PV067325.fasta
处理完成: PV067326.fasta
处理完成: PV067327.fasta
处理完成: PV067328.fasta
处理完成: PV067329.fasta
处理完成: PV067330.fasta
处理完成: PV067331.fasta
处理完成: PV067332.fasta
处理完成: PV067333.fasta
处理完成: PV067334.fasta
处理完成: PV067335.fasta
处理完成: PV067336.fasta
处理完成: PV067337.fasta
处理完成: PV067338.fasta
处理完成: PV067339.fasta
处理完成: PV067340.fasta
处理完成: PV067341.fasta
处理完成: PV067342.fasta
处理完成: PV067343.fasta
处理完成: PV06734

In [2]:
!for file in /mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列/*.fasta; do cat $file >> /mnt/c/Users/Administrator/Desktop/merge.fasta;done

In [4]:
!rm /mnt/f/OneDrive/文档（科研）/脚本/我的科研脚本/Python/数据科学/下载序列/*.fasta